# Solution of the "Off-Diagonal" team

**Process Management Problem**

In the process management problem, we are given a set of tasks represented by processes $p$. Each process $p_i$ has an associated value $v_i$​ (indicating its importance) and a duration $d_i$​ (representing the time required to execute the task). Additionally, there exists a maximum allowed duration $d_{max}$​ within which we must execute these tasks.


Our objective is to select a subset $S$ of processes from $p$ such that their total duration does not exceed $d_{max}$​, while maximizing the overall value of the executed processes. In other words, we aim to find the set S that maximizes the expression:

$$\sum_{i\in S} v_i$$

subject to the constraint:

$$\sum_{i\in S} d_i \leq d_{max}$$

## Define problem instance

In [1]:
import random 

random.seed(13)
# Define the number of items
n_items = 6

# Define ranges
duration_range = [1, 7]
values_range = [5, 15]
max_duration_percentage = 0.7

# Fill the weights and values 
duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))

# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [3, 3, 6, 6, 7, 7]
Values                   : [7, 15, 8, 15, 7, 8]
Total duration           : 32
Maximum allowed duration : 22


Reference Solution (Selected Processes are **Bolded**):

Duration: [3, **3**, **6**, **6**, 7, **7**]

Values: [7, **15**, **8**, **15**, 7, **8**]

Total selected Duration: 22

Total selected values: 46

# Hackathon Tasks 


## Problem 1

### Build the cost hamiltonians for the Process Management problem:

If there was no contraint of a maximum duration, the cost hamiltonian to minimize would simply be:

$$H_B= - \sum_{i} v_i \cdot b_i $$

where $b_i$ is 1 if the task is selected and 0 if not. But to punish the selections of tasks which don't satisfy the contraint, we add another Hamiltonian $H_A$ 

A solution to this problem was suggested in $[1]$ 

$$H_A=A*\left( \sum_{j=0}^{M-1} 2^j y_j + (c+1+2^M)y_M -\sum_{i=0}^{n-1}d_i b_i \right)^2$$

with $M = \lfloor\log_2(d_{\text{max}})\rfloor$, $y_i$ is the i-th helper qubit needed to evaluate perfectly if we don't satisfy the contraints. A is a constant which must be bigger than the maximum duration. Here we have added M + 1 new qubits to perform this calculation. To understand where this hamiltonian comes from nd hw these additional qubits help we look at an example of a set of tasks which respect the boundary condition and one which doesn't. 

If $\sum_{i=0}^{n-1}d_i b_i \leq d_{max}$ then there exist a binary encoding with M bits $(y_0, y_1, ... y_{M-1},y_M)$ where the binary values $y_j$ for $j \in \{1, M-1\}$ add the respective power of two $2^j$, not here that the maximum total duration we can encode with these first M qubits is $2^M-1$. Remembering that $M = \lfloor\log_2(d_{\text{max}})\rfloor$ we can conclude that $2^M \leq c \leq 2^{M+1}$ therefore the $y_M$ bit must be multiplied with the difference between $c$ and $2^M - 1$. Since the last bit of this string is not multiplied with a power of two the encoding is not necessarly unique qhich will produce degenerate ground state.

Therefore any state of the first $N qubits$ which satisfies the maximal duration condition will have at least one state in the $M+1$ qubit space which encodes this value in this pseudo-binary setting the cost Hamiltonian to zero. If the condition is not met there will never be such a pseudo-binary encoding $(y_0, y_1, ... y_{M-1},y_M)$ that matches the resulting total duration. Therefore $H_A$ will not be zero and these states are punished quadratically for their overshooting.   

Reference [1] claims this is a perfect implementation of the Knapsack problem, but it requires $n + log_2(d_{max})+1$ qubits.

$[1]  \href{https://arxiv.org/abs/1701.05584}{Mark W. Coffrey Adiabatic quantum computing solution of the knapsack problem }$

In [2]:
from qibo.symbols import Z, X
from qibo.hamiltonians import SymbolicHamiltonian
import numpy as np
import math

def build_cost_hamiltonian(values: list[int], duration: list[int], max_duration: int) -> SymbolicHamiltonian:
    """This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration.
        
    """
    B=1
    A=max(values)+1
    M=math.floor(np.log2(max_duration)) #round down the log2 of the max duration
    n=len(values)
    H_A_not_squared = sum( np.power(2,j)*(1-Z(j+n))/2 for j in range(0,M)) + (max_duration+1-np.power(2,M))*(1-Z(M+n))/2 - sum(duration[i]*(1-Z(i))/2 for i in range(0,len(values)))
    H_A = A*H_A_not_squared*H_A_not_squared
    H_B= - B*sum(values[i]*(1-Z(i))/2 for i in range(0,len(values)))
    
    cost_hamiltonian = H_A+H_B
    
    return SymbolicHamiltonian(cost_hamiltonian)

### Compute the number fo qubits required to execute the cost hamiltonian

In [3]:
# define the number of qubits:
nqubits=math.floor(np.log2(max_duration)) + len(values)+1

### Diagonalize the cost hamiltonian and examen its eigenvalues and eigenvectors. 
- Show that the ground state eigenvector does actually correspond to the ground state energy of the problem. (Note: the ground state could be degenerate)

In [4]:
import numpy as np
from qibo import set_backend

# set the backend used for the calculation 
set_backend("numpy", platform=None)


# create the cost Hamiltonian for the given graph
cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

ham_matrix = cost_hamiltonian.matrix

eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]

vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])

print()
print("first 3 eigenvectors: ",diagonalized_solution[0:3])

print("we found ",diagonalized_solution[0][1][0:len(values)], "by minimizing the hamiltonian, and the correct solution is 011101")

[Qibo 0.2.7|INFO|2024-05-05 11:12:06]: Using numpy backend on /CPU:0
[Qibo 0.2.7|WARNING|2024-05-05 11:12:06]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



first 3 eigenvectors:  [((-46+0j), '01110111111'), ((-45+0j), '01111011111'), ((-45+0j), '11010100111')]
we found  011101 by minimizing the hamiltonian, and the correct solution is 011101


## Problem 2

### Given that we only have **6 qubits** to solve this problem. Reduce the hamiltonian to use **only 6 qubits**.

We don't have any auxiliary qubits, we can hence only hope to find a good heuristic algorithm, which gives a good response most of the time.
To put the constraint into the Hamiltonian one would ideally add a exponential cost function which is small if $D \leq 0$ with $D := \left(\sum_{i=0}^{n-1}d_i b_i -d_{\text{max}} \right)$ and explodes with $D \geq 0$. Considering the fact that we will only be able to aply at most two qubit gates we an take the taylor expansion of this exponential up to second order. Finally one can then play around with the parameters of the linear and quadratic term to create an unbalenced cost function. We want to produce a positiv signed parabola which induces small cost for states with $D \approx 0$ so that we don't favor solutions which don't fully uses the maximal duration condition. However we want to punish more stricly states which are above $D > 0$. 

Therefor the cost hamiltonian looks like :

$$H_A = aD^2 + bD $$

To test our heuristic solution, we tested it on a randomly chosen batch of 500 knapsack problems

In [83]:
import knapsack #pip install knapsack
seeds=[]
N=500 # number of different priblems tested

for i in range(N):
    seeds.append(random.randint(1,10000))

values_matrix= []
duration_matrix=[]
max_duration_list=[]

for i in range(N):
    random.seed(seeds[i])
    duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(nqubits)]
    values  = [random.randint(values_range[0], values_range[1]) for _ in range(nqubits)]
    max_duration = int(max_duration_percentage * sum(duration))
    duration_matrix.append(duration)
    values_matrix.append(values)
    max_duration_list.append(max_duration)

nqubits = 6
correct_solutions=[] 
for i in range(N):

    result=knapsack.knapsack(duration_matrix[i], values_matrix[i]).solve(max_duration_list[i])[1]
    string_result=""
    for i in range(6):
        if i in result:
            string_result += "1"
        else:
            string_result+="0"

    correct_solutions.append(string_result)

print("correct solutions:",correct_solutions)

correct solutions: ['011111', '111011', '111010', '101110', '101111', '101110', '011111', '110111', '011111', '101101', '010111', '011101', '111110', '011101', '111100', '011101', '110011', '110011', '101011', '101111', '110110', '001111', '100111', '100111', '010111', '110011', '010111', '111110', '001111', '101101', '111101', '011101', '111100', '111110', '111011', '111110', '111010', '110110', '001111', '101011', '110111', '101110', '111011', '101110', '110111', '001111', '001111', '101011', '100111', '101101', '110011', '011101', '110101', '111011', '011101', '111011', '101111', '101011', '010111', '110111', '111110', '011011', '111010', '101110', '011110', '011110', '011110', '111001', '010111', '110111', '110110', '101110', '111001', '111011', '110110', '111011', '001111', '111001', '110011', '101101', '111011', '101110', '111101', '110011', '111001', '101111', '110101', '111010', '111100', '110101', '101111', '111100', '100111', '111110', '011101', '111001', '100111', '111001', 

We test different arguments a, b for the linear and quadratic terms of $H_A$ to maximize the amount of correct solutions

In [84]:
def percentage_of_correct_prediction(argument_array,solution_rank):   
    a = argument_array[0] #quadratic term
    b= argument_array[1] #linear term

    heuristic_solution=[]
    #additional cost a*D^2 + b*D
    nqubits_heuristic=len(values)
    for i in range(N):


        n=len(values)
        D = sum(duration_matrix[i][j]*(1-Z(j))/2 for j in range(0,n)) - max_duration_list[i]
        cost_hamiltonian_temp = -sum(values_matrix[i][j]*(1-Z(j))/2 for j in range(0,n)) + a*D*D + b*D  

        # Get prediction
        cost_hamiltonian = SymbolicHamiltonian(cost_hamiltonian_temp)
        ham_matrix = cost_hamiltonian.matrix
        eig_val, eig_vec = np.linalg.eig(ham_matrix)
        eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits_heuristic) for i in eig_vec]
        vec = zip(eig_val, eig_vec)
        diagonalized_solution = sorted(vec, key=lambda x: x[0])
        heuristic_solution.append(diagonalized_solution[solution_rank][1])
    tot = 0
    for i in range(N):
        if heuristic_solution[i]==correct_solutions[i]:
            tot+=1
    print("Percentage of correct answers in position",solution_rank,":",100*tot/N, "%")
    return 100*tot/N

By testing out a lot of different parameters for the linear and quadratic terms, our best proposition for $a$ and $b$ is $a=1$ and $ b = 4$

In [85]:
percentage_of_correct_prediction([1,4],0) #percentage of correct solution found

[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:33:43]: Calculating the dens

Percentage of correct answers in position 0 : 70.0 %


70.0

In [86]:
percentage_of_correct_prediction([1,4],1) #percentage of time when the correct solution is in second place

[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:01]: Calculating the dens

Percentage of correct answers in position 1 : 14.2 %


14.2

In [87]:
percentage_of_correct_prediction([1,4],2) #percentage of time when correct solution is in third place

[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:34:19]: Calculating the dens

Percentage of correct answers in position 2 : 8.2 %


8.2

As you can see this solution is correct around 70% of the time, and more than 90% of the time the perfect solution if one the 3 lowest eigenstate

### Solve this hamiltonian using the QAOA algorithm. 


We start with a seed where our heuristic hamiltonian indeed has the correct ground state

In [104]:
random.seed(2)
# Define the number of items
n_items = 6

# Define ranges
duration_range = [1, 7]
values_range = [5, 15]
max_duration_percentage = 0.7

# Fill the weights and values 
duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))

# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [7, 7, 1, 1, 1, 3]
Values                   : [7, 15, 9, 9, 14, 8]
Total duration           : 20
Maximum allowed duration : 14


So we implement the cost hamiltonian with the optimal parameters that we found for the given range of values

In [105]:
def build_cost_hamiltonian(values: list[int], duration: list[int], max_duration: int) -> SymbolicHamiltonian:
    """This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration.
        
    """
    a=1
    b=4
    n = len(values)
    D = sum(duration[i]*(1-Z(i))/2 for i in range(0,n)) - max_duration
    cost_hamiltonian = -sum(values[j]*(1-Z(j))/2 for j in range(0,n)) + a*D*D + b*D  
    
    return SymbolicHamiltonian(cost_hamiltonian)

We check that the given set of duration and values has indeed the correct solution as a ground state (which is the case in about 70% of cases) by comparing its prediction with a classical implementation of knapsack

In [106]:
#implement classic solver of knapsack to test batches of solution
def correct_knapsack(duration,values,max_duration):
    result=knapsack.knapsack(duration, values).solve(max_duration)[1]
    string_result=""
    for i in range(6):
        if i in result:
            string_result += "1"
        else:
            string_result+="0"
    print("The correct result is:",string_result)
    return string_result

In [107]:

# create the cost Hamiltonian for the given graph
cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
ham_matrix = cost_hamiltonian.matrix
eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])
print()
# print(diagonalized_solution)
print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
correct_result=correct_knapsack(duration,values,max_duration)

[Qibo 0.2.7|WARNING|2024-05-05 10:47:11]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



The ground state with the heuristic method is  011111
The correct result is: 011111


        step 1. Define the Mixing hamiltonian.

We use the sum of the $X(i)$ as mixer hamiltonian, to rotate a state in all directions. We can only use linear or quadratic terms (since physically we can only have interaction between two qubits at once on the physical setup), hence we have a sum of the $X(i)$ and not a product of these

In [108]:
def build_mixer_hamiltonian(nqubits: int) -> SymbolicHamiltonian:
    '''
    build the mixer hamiltonian for the given graph.

    args:
        graph: graph
            A network graph

    returns:
        The mixer hamiltonian of the given graph

    '''
    mixing_hamiltonian =  sum((X(i)) for i in range(nqubits)) 
    
    return SymbolicHamiltonian(mixing_hamiltonian)

        Step 2. Run the QAOA algorithm.

QAOA approximates the solution by iteratively applying the quantum circuit and a classical optimization.

In [109]:
from qibo import models

hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
mixer_hamiltonian = build_mixer_hamiltonian(nqubits)

# create QAOA model given the Hamiltonians
qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)
# Define random initial variational parameters with four layers 
n_layers = 4
initial_parameters = 0.01 * np.random.random(n_layers * 2)


"""
supported optimization Method: 
- Nelder-Mead 
- parallel_L-BFGS-B
- Powell
- CG
- cma
- sgd
- L-BFGS-B
- Newton-CG
- COBYLA
- BFGS
- trust-constr
"""
method = "Powell"


best_energy, final_parameters, _ = qaoa.minimize(initial_parameters, method=method)

print("best energy: ", best_energy)

best energy:  -35.013184920318686


    Step 3. Display solution and compare it to the ground state you obtained in problem 1

In [110]:
qaoa.set_parameters(final_parameters)
quantum_state = qaoa.execute(None)

probabilities = (np.abs(quantum_state) ** 2)
probabilities = zip(probabilities, [i for i in range(len(probabilities))])
probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
probabilities = [("{:.2f} %".format(round(p*100, 4)), "{0:0{bits}b}".format(s, bits=nqubits)) for p, s in probabilities]
print(probabilities)
qaoa_solution = probabilities[0][1]
print("-"*10)

print("The QAOA solution is:", qaoa_solution )
correct_knapsack(duration,values,max_duration)

[('10.38 %', '010111'), ('10.38 %', '011011'), ('10.37 %', '011101'), ('5.07 %', '010101'), ('5.07 %', '011001'), ('4.34 %', '101101'), ('4.31 %', '011111'), ('3.97 %', '100111'), ('3.97 %', '101011'), ('3.63 %', '011100'), ('3.58 %', '010011'), ('2.66 %', '101001'), ('2.66 %', '100101'), ('2.51 %', '111100'), ('2.26 %', '110010'), ('2.10 %', '100011'), ('1.82 %', '010000'), ('1.80 %', '101111'), ('1.45 %', '111111'), ('1.38 %', '101100'), ('1.37 %', '010110'), ('1.37 %', '011010'), ('1.34 %', '010010'), ('1.27 %', '100000'), ('1.12 %', '110000'), ('0.99 %', '010001'), ('0.73 %', '100110'), ('0.73 %', '101010'), ('0.61 %', '100001'), ('0.57 %', '100010'), ('0.51 %', '001100'), ('0.51 %', '111010'), ('0.51 %', '110110'), ('0.50 %', '111000'), ('0.50 %', '110100'), ('0.48 %', '111001'), ('0.48 %', '110101'), ('0.43 %', '000011'), ('0.37 %', '111110'), ('0.30 %', '111101'), ('0.30 %', '001111'), ('0.25 %', '110011'), ('0.17 %', '001110'), ('0.11 %', '000010'), ('0.09 %', '110111'), ('0.09

'011111'

We do not converge all the time to the correct solution. For example with the seed 2, the good solution is only in the 6th spot.

We first try out the different optimizer methods to see if some of them arre more likely to converge to the good solution for our problem

In [38]:
from qibo import models

random.seed(92)
n_items = 6
duration_range = [1, 7]
values_range = [5, 15]
max_duration_percentage = 0.7
duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]
max_duration = int(max_duration_percentage * sum(duration))

cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
ham_matrix = cost_hamiltonian.matrix
eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])
print()
print(diagonalized_solution)
print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
correct_result=correct_knapsack(duration,values,max_duration)

if correct_result == diagonalized_solution[0][1][0:len(values)]:
    print("results match")

    liste_methods = ['Nelder-Mead','parallel_L-BFGS-B','Powell','CG','L-BFGS-B','COBYLA','BFGS','trust-constr']
    for i in range(len(liste_methods)):
        hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
        mixer_hamiltonian = build_mixer_hamiltonian(nqubits)
        # create QAOA model given the Hamiltonians
        qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)
        # optimize using random initial variational parameters with four layers 
        n_layers = 4
        initial_parameters = 0.01 * np.random.random(n_layers * 2)
        # initial_parameters =  0.01 *  (2 * np.random.random(n_layers * 2) - 1) * np.pi

        method = liste_methods[i]
        best_energy, final_parameters, _ = qaoa.minimize(initial_parameters, method=method)
        qaoa.set_parameters(final_parameters)
        quantum_state = qaoa.execute(None)

        probabilities = (np.abs(quantum_state) ** 2)
        probabilities = zip(probabilities, [i for i in range(len(probabilities))])
        probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
        probabilities = [("{:.2f} %".format(round(p*100, 4)), "{0:0{bits}b}".format(s, bits=nqubits)) for p, s in probabilities]
        #print(probabilities)
        qaoa_solution = probabilities[0][1]

        print(liste_methods[i],"qaoa solution is:", qaoa_solution )
        print("-"*100)
        print()

[Qibo 0.2.7|WARNING|2024-05-05 08:11:24]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



[((-56+0j), '101011'), ((-56+0j), '111010'), ((-53+0j), '101110'), ((-51+0j), '111001'), ((-50+0j), '110011'), ((-48+0j), '101101'), ((-48+0j), '111100'), ((-47+0j), '011110'), ((-47+0j), '100111'), ((-47+0j), '110110'), ((-46+0j), '011011'), ((-43+0j), '001111'), ((-42+0j), '001011'), ((-42+0j), '110101'), ((-38+0j), '011101'), ((-37+0j), '010111'), ((-37+0j), '011001'), ((-37+0j), '101001'), ((-37+0j), '111110'), ((-36+0j), '010011'), ((-36+0j), '100011'), ((-34+0j), '001101'), ((-33+0j), '000111'), ((-31+0j), '110001'), ((-28+0j), '010101'), ((-28+0j), '100101'), ((-26+0j), '011010'), ((-23+0j), '001110'), ((-22+0j), '101010'), ((-20+0j), '111011'), ((-18+0j), '011100'), ((-17+0j), '010110'), ((-17+0j), '101111'), ((-17+0j), '111000'), ((-16+0j), '110010'), ((-14+0j), '101100'), ((-13+0j), '100110'), ((-12+0j), '111101'), ((-11+0j), '110111'), ((-8+0j), '110100'), ((3+0j), '011111'), ((17+0j), '001001'), ((18+0j), '000011'), ((23+0j), '010001'), ((26+0j), '000101'), ((33+0j), '1000

Different optimizers lead to different results, either because they have not converged yet, or because they are stuck in local minima.

To determine which optimizer performs best, we ttried a batch of problems with all the solvers and counted how many problems were correctly solved by each optimizer

In [57]:
liste_methods = ['Nelder-Mead','parallel_L-BFGS-B','Powell','CG','L-BFGS-B','COBYLA','BFGS','trust-constr']
models_score= np.zeros(len(liste_methods))
for j in range(30):
    random.seed(j)
    n_items = 6
    duration_range = [1, 7]
    values_range = [5, 15]
    max_duration_percentage = 0.7
    duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
    values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]
    max_duration = int(max_duration_percentage * sum(duration))

    cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
    ham_matrix = cost_hamiltonian.matrix
    eig_val, eig_vec = np.linalg.eig(ham_matrix)
    eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
    vec = zip(eig_val, eig_vec)
    diagonalized_solution = sorted(vec, key=lambda x: x[0])
    print()
    print(diagonalized_solution)
    print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
    correct_result=correct_knapsack(duration,values,max_duration)

    if correct_result == diagonalized_solution[0][1][0:len(values)]:
        print("results match")

 
        for i in range(len(liste_methods)):
            hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
            mixer_hamiltonian = build_mixer_hamiltonian(nqubits)
            # create QAOA model given the Hamiltonians
            qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)
            # optimize using random initial variational parameters with four layers 
            n_layers = 4
            initial_parameters = 0.01 * np.random.random(n_layers * 2)
            # initial_parameters =  0.01 *  (2 * np.random.random(n_layers * 2) - 1) * np.pi

            method = liste_methods[i]
            best_energy, final_parameters, _ = qaoa.minimize(initial_parameters, method=method)
            qaoa.set_parameters(final_parameters)
            quantum_state = qaoa.execute(None)

            probabilities = (np.abs(quantum_state) ** 2)
            probabilities = zip(probabilities, [i for i in range(len(probabilities))])
            probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
            probabilities = [("{:.2f} %".format(round(p*100, 4)), "{0:0{bits}b}".format(s, bits=nqubits)) for p, s in probabilities]
            #print(probabilities)
            qaoa_solution = probabilities[0][1]

            print(liste_methods[i],"qaoa solution is:", qaoa_solution )
            print("-"*100)
            print()

            if qaoa_solution == correct_result:
                models_score[i]+=1


In [40]:
#print methods score
for i in range(len(liste_methods)):
    print(liste_methods[i], "was correct in",models_score[i], "cases")

Nelder-Mead was correct in 10.0 cases
parallel_L-BFGS-B was correct in 4.0 cases
Powell was correct in 4.0 cases
CG was correct in 6.0 cases
L-BFGS-B was correct in 7.0 cases
COBYLA was correct in 1.0 cases
BFGS was correct in 2.0 cases
trust-constr was correct in 2.0 cases


We hence decided to use the Nedler-Mead optimizer method. We then investigated which initial parameters could improve the number of correct solutions. The best performance that we achieved was setting n_layers to 8. In this case, out of the 50 cases tested, 50% of the groundsates were found by QAOA. This is however not optimal.

In [47]:

nb_correct_result = 0
calculated_count=0
for j in range(100,150):

    random.seed(j)
    n_items = 6
    duration_range = [1, 7]
    values_range = [5, 15]
    max_duration_percentage = 0.7
    duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
    values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]
    max_duration = int(max_duration_percentage * sum(duration))

    cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
    ham_matrix = cost_hamiltonian.matrix
    eig_val, eig_vec = np.linalg.eig(ham_matrix)
    eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
    vec = zip(eig_val, eig_vec)
    diagonalized_solution = sorted(vec, key=lambda x: x[0])
    print()
    print(diagonalized_solution)
    print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
    correct_result=correct_knapsack(duration,values,max_duration)

    if correct_result == diagonalized_solution[0][1][0:len(values)]:
        calculated_count+=1
        print("results match")
        hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
        mixer_hamiltonian = build_mixer_hamiltonian(nqubits)
        # create QAOA model given the Hamiltonians
        qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)
        # optimize using random initial variational parameters with four layers 
        n_layers = 8
        initial_parameters = 0.01 * np.random.random(n_layers * 2)
        # initial_parameters =  0.01 *  (2 * np.random.random(n_layers * 2) - 1) * np.pi
        method = 'Nelder-Mead'
        best_energy, final_parameters, _ = qaoa.minimize(initial_parameters,initial_state=initial_state, method=method)
        qaoa.set_parameters(final_parameters)
        quantum_state = qaoa.execute(None)

        probabilities = (np.abs(quantum_state) ** 2)
        probabilities = zip(probabilities, [i for i in range(len(probabilities))])
        probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
        probabilities = [("{:.2f} %".format(round(p*100, 4)), "{0:0{bits}b}".format(s, bits=nqubits)) for p, s in probabilities]
        #print(probabilities)
        qaoa_solution = probabilities[0][1]

        print("qaoa solution is:", qaoa_solution )
        if qaoa_solution  == correct_result:
            print("correct qaoa")
            nb_correct_result+=1
        print("-"*100)

print("correct result:",nb_correct_result)
print("calculated cases:",calculated_count)


### Solve this hamiltonian using Quantum Adiabatic Evolution

The Quantum Adiabatic Evolution seems more promising, with 100% of the groundstates found on a batch of 12 randomly chosen problems

        Setp 1. Define initial hamiltonian

In [71]:
def build_initial_hamiltonian(nqubits: int) -> SymbolicHamiltonian:

    mixing_hamiltonian = -sum((X(i)) for i in range(nqubits))  
    
    return SymbolicHamiltonian(mixing_hamiltonian)

    Step 2. Run the Adiabatic Evolution

In [79]:
import numpy as np
from qibo.models.evolution import AdiabaticEvolution
from qibo.hamiltonians.hamiltonians import Hamiltonian


# build initial (H0) and target (H1) hamiltonians
H0 = build_initial_hamiltonian(nqubits=nqubits)
H1 = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

# calculate the dense hamiltonian from the symbolic hamiltonian
H1_dense = Hamiltonian(nqubits, H1.calculate_dense().matrix)
H0_dense = Hamiltonian(nqubits, H0.calculate_dense().matrix)

# Define the time steps
dt = 0.1
# Define the final evolution time
T = 10

# define the schedule. This is a function of time that defines the scheduling of the adiabatic evolution. 
# Can be either a function of time s(t) or a function with two arguments s(t, p) 
# where p corresponds to a vector of parameters to be optimized.
def s(t): return t

# construct the adiabatic model
adiabatic_model = AdiabaticEvolution(H0_dense, H1_dense, s, dt)

# execute the adiabatic optimization
adiabatic_quantum_state = adiabatic_model.execute(final_time=T)

    Step 3. Display solution and compare it to the ground state you obtained in problem 1 and using QAOA

In [80]:
# obtain the state with the highest probability
solution_dec  = (np.abs(adiabatic_quantum_state) ** 2).argmax()
adiabatic_solution = "{0:0{bits}b}".format(solution_dec, bits=nqubits)


print("the adiabatic evolution solution is: ", adiabatic_solution)
correct_knapsack(duration,values,max_duration)

the adiabatic evolution solution is:  111001
The correct result is: 111001


'111001'

The variational method with dt =0.1 and T=10 works 100% of the 12 randomly chosen tests. With a correct solution each time that the ground state of the heuristic hamiltonian indeed the correct solution was. So if our heurisitc is good, the adiabatic evolution will almost always find the solution.

In [81]:

nb_correct_result = 0
calculated_count=0
for j in range(100,115):

    random.seed(j)
    n_items = 6
    duration_range = [1, 7]
    values_range = [5, 15]
    max_duration_percentage = 0.7
    duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
    values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]
    max_duration = int(max_duration_percentage * sum(duration))

    cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
    ham_matrix = cost_hamiltonian.matrix
    eig_val, eig_vec = np.linalg.eig(ham_matrix)
    eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
    vec = zip(eig_val, eig_vec)
    diagonalized_solution = sorted(vec, key=lambda x: x[0])
    print()
    print(diagonalized_solution)
    print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
    correct_result=correct_knapsack(duration,values,max_duration)

    if correct_result == diagonalized_solution[0][1][0:len(values)]:
        calculated_count+=1
        print("results match")
        H0 = build_initial_hamiltonian(nqubits=nqubits)
        H1 = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

        # calculate the dense hamiltonian from the symbolic hamiltonian
        H1_dense = Hamiltonian(nqubits, H1.calculate_dense().matrix)
        H0_dense = Hamiltonian(nqubits, H0.calculate_dense().matrix)
        # calculate the ground state with adiabatic evolution
        # construct the adiabatic model
        adiabatic_model = AdiabaticEvolution(H0_dense, H1_dense, s, dt)

        # execute the adiabatic optimization
        adiabatic_quantum_state = adiabatic_model.execute(final_time=T)
        # obtain the state with the highest probability
        solution_dec  = (np.abs(adiabatic_quantum_state) ** 2).argmax()
        adiabatic_solution = "{0:0{bits}b}".format(solution_dec, bits=nqubits)
        print("the adiabatic evolution solution is: ", adiabatic_solution)
        
        if adiabatic_solution  == correct_result:
            print("correct adiabatic")
            nb_correct_result+=1
        print("-"*100)

print("correct result:",nb_correct_result)
print("calculated cases:",calculated_count)

[Qibo 0.2.7|WARNING|2024-05-05 10:03:23]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



[((-48+0j), '111001'), ((-46+0j), '111011'), ((-45+0j), '101110'), ((-45+0j), '111100'), ((-44+0j), '101011'), ((-44+0j), '110110'), ((-43+0j), '011011'), ((-43+0j), '100111'), ((-43+0j), '110011'), ((-41+0j), '100101'), ((-40+0j), '011110'), ((-39+0j), '111110'), ((-38+0j), '011100'), ((-37+0j), '001101'), ((-37+0j), '011001'), ((-36+0j), '000111'), ((-36+0j), '010101'), ((-36+0j), '101101'), ((-35+0j), '101100'), ((-35+0j), '110101'), ((-34+0j), '110100'), ((-33+0j), '111010'), ((-31+0j), '001111'), ((-30+0j), '001110'), ((-30+0j), '010111'), ((-30+0j), '101001'), ((-29+0j), '010110'), ((-29+0j), '110001'), ((-26+0j), '000101'), ((-25+0j), '001011'), ((-24+0j), '010011'), ((-22+0j), '101111'), ((-21+0j), '110111'), ((-18+0j), '100110'), ((-15+0j), '011101'), ((-12+0j), '001100'), ((-11+0j), '010100'), ((-11+0j), '111000'), ((-9+0j), '100011'), ((-6+0j), '011010'), ((-3+0j), '001001'), ((-2+0j), '010001'), ((2+0j), '111101'), ((7+0j), '011111'), ((8+0j), '100100'), ((13+0j), '000110'

[Qibo 0.2.7|WARNING|2024-05-05 10:03:51]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


the adiabatic evolution solution is:  111001
correct adiabatic
----------------------------------------------------------------------------------------------------

[((-54+0j), '011011'), ((-53+0j), '011110'), ((-50+0j), '111010'), ((-44+0j), '011101'), ((-43+0j), '001111'), ((-43+0j), '101110'), ((-43+0j), '101111'), ((-41+0j), '111001'), ((-40+0j), '010110'), ((-40+0j), '010111'), ((-40+0j), '101011'), ((-38+0j), '100111'), ((-38+0j), '101101'), ((-37+0j), '010011'), ((-37+0j), '011111'), ((-37+0j), '110010'), ((-37+0j), '110011'), ((-36+0j), '011100'), ((-36+0j), '111100'), ((-35+0j), '010101'), ((-34+0j), '111011'), ((-33+0j), '111000'), ((-32+0j), '110001'), ((-31+0j), '011001'), ((-31+0j), '110100'), ((-30+0j), '011010'), ((-30+0j), '110110'), ((-23+0j), '111110'), ((-22+0j), '100110'), ((-21+0j), '100101'), ((-18+0j), '000111'), ((-15+0j), '100011'), ((-15+0j), '110101'), ((-14+0j), '101100'), ((-11+0j), '010100'), ((-8+0j), '001101'), ((-8+0j), '110000'), ((-5+0j), '101001'), (

[Qibo 0.2.7|WARNING|2024-05-05 10:04:20]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:04:20]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


...

the adiabatic evolution solution is:  101101
correct adiabatic
----------------------------------------------------------------------------------------------------

[((-55+0j), '011111'), ((-51+0j), '010111'), ((-51+0j), '011101'), ((-48+0j), '110110'), ((-45+0j), '001111'), ((-44+0j), '011110'), ((-44+0j), '110011'), ((-44+0j), '110100'), ((-43+0j), '111010'), ((-42+0j), '111100'), ((-40+0j), '011011'), ((-40+0j), '110001'), ((-39+0j), '110101'), ((-38+0j), '100110'), ((-38+0j), '111001'), ((-37+0j), '100101'), ((-36+0j), '101100'), ((-36+0j), '101110'), ((-35+0j), '010101'), ((-35+0j), '111000'), ((-34+0j), '100011'), ((-34+0j), '111110'), ((-33+0j), '001101'), ((-33+0j), '100111'), ((-33+0j), '110010'), ((-32+0j), '101001'), ((-32+0j), '101011'), ((-30+0j), '111011'), ((-29+0j), '000111'), ((-29+0j), '101010'), ((-27+0j), '110111'), ((-26+0j), '100100'), ((-24+0j), '011100'), ((-23+0j), '101101'), ((-22+0j), '100001'), ((-20+0j), '011001'), ((-18+0j), '001110'), ((-16+0j), '010110')

[Qibo 0.2.7|WARNING|2024-05-05 10:08:10]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


the adiabatic evolution solution is:  011111
correct adiabatic
----------------------------------------------------------------------------------------------------

[((-49+0j), '111001'), ((-48+0j), '101011'), ((-47+0j), '101110'), ((-45+0j), '101101'), ((-45+0j), '110011'), ((-44+0j), '001111'), ((-44+0j), '011011'), ((-43+0j), '111010'), ((-42+0j), '110101'), ((-41+0j), '011101'), ((-40+0j), '101111'), ((-40+0j), '110110'), ((-40+0j), '111100'), ((-39+0j), '010111'), ((-39+0j), '011010'), ((-38+0j), '111000'), ((-37+0j), '100111'), ((-36+0j), '011100'), ((-31+0j), '011110'), ((-29+0j), '011001'), ((-28+0j), '010110'), ((-28+0j), '110010'), ((-27+0j), '001110'), ((-27+0j), '110111'), ((-25+0j), '101010'), ((-25+0j), '110100'), ((-24+0j), '111011'), ((-22+0j), '101100'), ((-21+0j), '111101'), ((-17+0j), '010011'), ((-14+0j), '010101'), ((-12+0j), '001011'), ((-10+0j), '110001'), ((-9+0j), '001101'), ((-8+0j), '100110'), ((-6+0j), '011111'), ((-3+0j), '101001'), ((5+0j), '111110'), ((6+

[Qibo 0.2.7|WARNING|2024-05-05 10:08:38]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-05 10:08:38]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


the adiabatic evolution solution is:  111001
correct adiabatic
----------------------------------------------------------------------------------------------------

[((-42+0j), '101101'), ((-41+0j), '111001'), ((-41+0j), '111010'), ((-40+0j), '111100'), ((-39+0j), '011011'), ((-38+0j), '011100'), ((-38+0j), '110101'), ((-38+0j), '110110'), ((-37+0j), '011010'), ((-37+0j), '101011'), ((-37+0j), '111011'), ((-36+0j), '010111'), ((-36+0j), '101110'), ((-34+0j), '001101'), ((-34+0j), '001110'), ((-34+0j), '010110'), ((-34+0j), '011101'), ((-34+0j), '100111'), ((-34+0j), '110111'), ((-31+0j), '011001'), ((-31+0j), '110011'), ((-30+0j), '001111'), ((-30+0j), '111101'), ((-28+0j), '010101'), ((-27+0j), '001011'), ((-26+0j), '101111'), ((-24+0j), '000111'), ((-19+0j), '010011'), ((-16+0j), '101100'), ((-9+0j), '111000'), ((-6+0j), '110100'), ((-5+0j), '101010'), ((-2+0j), '001100'), ((-2+0j), '100110'), ((2+0j), '011110'), ((7+0j), '011000'), ((7+0j), '110010'), ((10+0j), '010100'), ((11+0j), 

### What are the downsides to the method you used to reduce the number of qubits? 

Using only 6 qubits for 6 tasks, means that we cannot anymore implement a perfect solution, but rather a heuristic algorithm which is not garanteed to succeed. We cannot punish perfectly all the forbiden states with:
$$H_A = aD^2 + bD $$
So the heuristic algorithm might sometimes propose solutions which do not satisfy the constraints. Also since it might be that the maximum of $H_A$ lies at a distance which is not the distance of the optimal solution, the ground state might be a solution satisfying the constraints, but which is not optimal.

## Problem 3 NOT SOLVED ON THIS Notebook

### Given this new dataset generate a hamiltonian that only uses 3 qubits 

In [4]:
import random 


random.seed(13)

# Define the number of items
n_items = 3

# Define ranges
max_duration_percentage = 0.7

# Fill the weights and values 
values   = [6, 2, 6]
duration = [3, 3, 6]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))



# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [3, 3, 6]
Values                   : [6, 2, 6]
Total duration           : 12
Maximum allowed duration : 8


We inspire ourself from the design with 6 qubits for 6 tasks

In [ ]:
def build_cost_hamiltonian(values: list[int], duration: list[int], max_duration: int) -> SymbolicHamiltonian:
    """This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration.
        
    """
    a=1
    b=4
    n = len(values)
    D = sum(duration[i]*(1-Z(i))/2 for i in range(0,n)) - max_duration
    cost_hamiltonian = -sum(values[j]*(1-Z(j))/2 for j in range(0,n)) + a*D*D + b*D  
    
    return SymbolicHamiltonian(cost_hamiltonian)

In [ ]:
# create the cost Hamiltonian for the given graph
cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
ham_matrix = cost_hamiltonian.matrix
eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nqubits) for i in eig_vec]
vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])
print()
print(diagonalized_solution)
print("The ground state with the heuristic method is ",diagonalized_solution[0][1][0:len(values)])
correct_result=correct_knapsack(duration,values,max_duration)

### Design a circuit layout that is suitable for this Hamiltonian

    Step 1. Create each element of the circuit

Your qubits will have a critical current of 230nA, shunting capacitance of 50fF, Z loop capacitance of 4.4fF, inductance of 480pH, x to z loop current ration of 0.4 and junction assymetry of 0.

The coupler's parameters are given

In [ ]:
import cas as cas

# coupler parameters
i_sigma = 565
c_sigma = 11
lc = 580
d = 0.0

# define couplers and qubits

circuit_elements = [...]

    Step 2. Define the qubit-coupler interaction and create the circuit object

A good approximation is to consider only the first 5 energy levels of the qubit and the first 2 energy levels of the coupler

In [ ]:
m = 65; m_mat = np.zeros((6, 6)) # mutual inductance matrix (given)

m_mat[0, 1] = -m; m_mat[1, 2] = m
m_mat[2, 3] = -m; m_mat[3, 4] = m
m_mat[4, 5] = -m; m_mat[5, 0] = m

m_mat = -(m_mat + m_mat.T)

trunc_vec = np.array([...]) # truncation vector for the energies of each element
circuit = cas.AnnealingCircuit(circuit_elements, m_mat, trunc_vec)

### Design an annealing Ising schedule 

    Step 1. Define the starting (initial) and objective hamiltonian coefficients

In [ ]:
# initialization hamiltonian
h0 = ... # type: dict[str, np.ndarray]

# objective hamiltonian
hf = ... # type: dict[str, np.ndarray]

    Step 2. Create a method to get a an arbitrary schedule for the circuit designed above

In [ ]:
def get_schedule(h0: dict[str, np.ndarray], hf: dict[str, np.ndarray], points: int, schedule: callable) -> dict[str, int | np.ndarray]:
    """Given the ising coefficients for the initial and final hamiltonian of the annealing scheduel,
    and given some time dependance of the schedule (eg. linear, exponential, etc), returns the ising
    coefficients for x and z at each time step.

    args:
        h0 (dict[str, np.ndarray]): ising coefficients of the starting hamiltonian h0
        hf (dict[str, np.ndarray]): ising coefficients of the target (final) hamiltonian hf
        points (int): number of points in the annealing schedule
        schedule (callable): time dependance of the schedule
    
    returns:
        dict[str, np.ndarray]: Dictionary of ising coefficients at each step of the schedule
    """

    ising_dict = ...
    
    return ...



    Step 2. Get the ising coefficients for a linear and an exponential schedule for a resolution of 1ns

In [ ]:
# linear schedule
linear_schedule = ... # feel free to use any callable (e.g. lambda function or method)
ising_linear = get_schedule(h0, hf, points=10, schedule=linear_schedule)
# exponential schedule
...

    Step 3. Calculate the flux schedule for the exponential schedule

In [ ]:
def get_fluxes(ising_schedule: dict[str, int | np.ndarray], optimizer: str) ->  dict[str, int | np.ndarray]:
    """Get the flux schedule from a given ising schedule.

    Args:
        ising_schedule (dict[str, int  |  np.ndarray]): dictionary with the number of points in the schedule
            and the ising coefficients for each control of the circuit at each point of the schedule
        optimizer (str): name of the lmfit minimizer to use

    Returns:
        dict[str, int | np.ndarray]: dictionary containing the number of points in the schedule and the fluxes
            to be applied at each control of the circuit at each point of the schedule.
    """
    flux_dict = ...
    return flux_dict
    

### With the obtain fluxes calculate the energy spectrum of the circuit

    Step 1. Define a function to get the spectrum from the flux schedule using the code from CAS

In [ ]:
def get_schedule_spectrum(flux_dict: dict[str, int | np.ndarray], levels: int) -> np.ndarray:
    """Calculate the energy spectrum through the schedule. The more levels we try to calculate,
    the more computationally expensive this will be.

    Args:
        flux_dict (dict[str, int  |  np.ndarray]): dictionary containing the number of points in the schedule and the fluxes
            to be applied at each control of the circuit at each point of the schedule.
        levels (int): energy levels to compute

    Returns:
        np.ndarray: of size (schedule points, levels) with the energy gaps for each level (En - E0),
            where E0 is the ground state energy and En is the energy of level n.
    """
    

### Research questions

1. For the annealing schedules calculated, do any of the resulting spectral schedules fulfill the adiabatic theorem?
2. By this point you will have obtained two energies, one from the software challenge (QAOA) and one from the annealing schedule above. Argue their relation.
3.  If you have made it here, you have problaly seen the difficulties of solving small instances. Can you propose ways to scale these procedures to more qubits? Some techniques for finding the gap along the annealing process can be found in [1], [2], [3] and [4]. Can you implement a simulation from one of these papers? Research and propose better techniques to find the flux schedule for a given Ising schedule with a greater amount of qubits.  

## Bibliography

[1] Adiabatic Spectroscopy and a Variational Quantum Adiabatic Algorithm: https://arxiv.org/abs/2103.01226

[2] Direct estimation of the energy gap between the ground state and excited state with quantum annealing: https://arxiv.org/abs/2007.10561

[3] Simulating quantum circuits by adiabatic computation: improved spectral gap bounds: https://arxiv.org/abs/1906.05233

[4] Spectroscopy on two coupled flux qubits: https://arxiv.org/abs/cond-mat/0308192

